In [77]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

In [78]:

df = pd.read_sql('select * from category',con=connection)
print(df)

    category_id         name         last_update
0             1       Action 2006-02-15 09:46:27
1             2    Animation 2006-02-15 09:46:27
2             3     Children 2006-02-15 09:46:27
3             4     Classics 2006-02-15 09:46:27
4             5       Comedy 2006-02-15 09:46:27
5             6  Documentary 2006-02-15 09:46:27
6             7        Drama 2006-02-15 09:46:27
7             8       Family 2006-02-15 09:46:27
8             9      Foreign 2006-02-15 09:46:27
9            10        Games 2006-02-15 09:46:27
10           11       Horror 2006-02-15 09:46:27
11           12        Music 2006-02-15 09:46:27
12           13          New 2006-02-15 09:46:27
13           14       Sci-Fi 2006-02-15 09:46:27
14           15       Sports 2006-02-15 09:46:27
15           16       Travel 2006-02-15 09:46:27


1.Ile kategorii filmów mamy w wypożyczalni?

In [79]:
print(len(pd.unique(df['category_id'])))

16


2.Wyświetl listę kategorii w kolejności alfabetycznej.

In [80]:
df2 = pd.read_sql('select name from category order by name ASC',con=connection)
print(df2)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


3.Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [89]:
df3 = pd.read_sql('select title, release_year from film order by release_year ASC',con=connection)
print(df3['release_year'][0],df3['title'][0])
print(df3['release_year'][len(df3)-1],df3['title'][len(df3)-1])


2006 Chamber Italian
2006 Zorro Ark


4.Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [82]:
df4 = pd.read_sql("select COUNT(*) from rental where rental_date between '2005-07-01'::timestamp and '2005-08-01'::timestamp",con=connection)
print(df4)

   count
0   6709


5.Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [83]:
df5 = pd.read_sql("select COUNT(*) from rental where rental_date between '2010-01-01'::timestamp and '2011-02-01'::timestamp",con=connection)
print(df5)

   count
0      0


6.Znajdź największą płatność wypożyczenia.

In [84]:
df6 = pd.read_sql("select MAX(amount) from payment",con=connection)
print(df6)

     max
0  11.99


7.Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [85]:
df7 = pd.read_sql("select customer.first_name,customer.last_name,country.country from (((customer inner join address on customer.address_id=address.address_id) inner join city on address.city_id=city.city_id) inner join country on city.country_id=country.country_id) where country in ('Poland','Nigeria','Bangladesh') ",  con=connection)
print(df7)

   first_name  last_name     country
0       Sonia    Gregory     Nigeria
1       Brian      Wyman      Poland
2      Sidney   Burleson      Poland
3      Rodney    Moeller     Nigeria
4     Stephen     Qualls  Bangladesh
5       Velma      Lucas     Nigeria
6     Marilyn       Ross     Nigeria
7       Elsie     Kelley     Nigeria
8      Gladys   Hamilton     Nigeria
9       Frank   Waggoner  Bangladesh
10   Marjorie     Tucker      Poland
11      Carol     Garcia     Nigeria
12       Leah     Curtis      Poland
13      Ruben      Geary      Poland
14       Olga    Jimenez     Nigeria
15     Bertha   Ferguson     Nigeria
16     Tracey    Barrett     Nigeria
17         Jo     Fowler     Nigeria
18    Johnnie   Chisholm      Poland
19    Wallace      Slone     Nigeria
20   Michelle      Clark  Bangladesh
21    Russell    Brinson      Poland
22     Jimmie  Eggleston      Poland
23  Constance       Reid     Nigeria


8.Gdzie mieszkają członkowie personelu?

In [86]:
df8 = pd.read_sql("select staff.first_name, staff.last_name,city.city,address.address,country.country from (((staff inner join address on staff.address_id=address.address_id) inner join city on address.city_id=city.city_id) inner join country on city.country_id=country.country_id)",con = connection)
print(df8)

  first_name last_name        city               address    country
0       Mike   Hillyer  Lethbridge     23 Workhaven Lane     Canada
1        Jon  Stephens   Woodridge  1411 Lillydale Drive  Australia


9.Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [87]:
df9 = pd.read_sql("select staff.first_name, staff.last_name,city.city,address.address,country.country from (((staff inner join address on staff.address_id=address.address_id) inner join city on address.city_id=city.city_id) inner join country on city.country_id=country.country_id) where country in ('Argentina','Spain')",con = connection)
print(len(df9))

0


10.Jakie kategorie filmów zostały wypożyczone przez klientów?

In [93]:
df11 = pd.read_sql("SELECT DISTINCT category.name from (((inventory inner join film on inventory.film_id=film.film_id) inner join film_category on film.film_id=film_category.film_id) inner join category on film_category.category_id=category.category_id)",con=connection)
print(df11)


           name
0        Horror
1        Comedy
2         Drama
3        Sci-Fi
4       Foreign
5      Classics
6         Games
7        Travel
8           New
9         Music
10       Action
11  Documentary
12       Sports
13    Animation
14     Children
15       Family


11.Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [95]:
df11 = pd.read_sql("SELECT DISTINCT category.name from \
    ((((((((country inner join city on country.country_id=city.country_id)\
    inner join address on city.city_id=address.city_id)\
    inner join customer on address.address_id=customer.customer_id)\
    inner join rental on customer.customer_id=rental.customer_id)\
    inner join inventory on rental.inventory_id=inventory.inventory_id)\
    inner join film on inventory.film_id=film.film_id)\
    inner join film_category on film.film_id=film_category.film_id)\
    inner join category on film_category.category_id=category.category_id) where country in ('United States')\
    ",con=connection)
print(df11)

           name
0        Horror
1        Comedy
2        Sci-Fi
3         Drama
4       Foreign
5      Classics
6         Games
7           New
8        Travel
9         Music
10       Action
11  Documentary
12       Sports
13    Animation
14       Family
15     Children


12.Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [97]:
df12 = pd.read_sql("SELECT actor.first_name,actor.last_name,film.title FROM ((film inner join film_actor on film.film_id=film_actor.film_id) inner join actor on film_actor.actor_id=actor.actor_id) WHERE first_name in ('Olympia','Julia','Ellen') and last_name in ('Pfeiffer','Zellweger','Presley')",con=connection)
print(df12)

   first_name  last_name              title
0       Ellen    Presley    Bilko Anonymous
1       Ellen    Presley  Caribbean Liberty
2       Ellen    Presley   Casper Dragonfly
3       Ellen    Presley   Empire Malkovich
4       Ellen    Presley      Floats Garden
..        ...        ...                ...
64      Julia  Zellweger       Outlaw Hanky
65      Julia  Zellweger    Panky Submarine
66      Julia  Zellweger   Rider Caddyshack
67      Julia  Zellweger          Won Dares
68      Julia  Zellweger      Wyoming Storm

[69 rows x 3 columns]
